In [1]:
from nltk.tag.perceptron import PerceptronTagger
from sklearn.model_selection import train_test_split
import nltk

sentences = nltk.corpus.treebank.tagged_sents()

train_sentences, test_sentences = train_test_split(sentences, test_size=0.25)

model = PerceptronTagger()
model.train(train_sentences)

print("Model score = ", model.evaluate(test_sentences))

Model score =  0.9705241472207916


In [2]:
from nltk.tag.hmm import HiddenMarkovModelTrainer

trainer = HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_sentences)

print("Model score = ", tagger.evaluate(test_sentences))

Model score =  0.3814428905352403


In [3]:
from nltk.tag.crf import CRFTagger

model = CRFTagger()
model.train(train_sentences, 'model.bin')

print("Model score = ", model.evaluate(test_sentences))

Model score =  0.9463174993066835
